In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import torch

In [ ]:
# llama="../output/final/layer_type_ratio_huggyllama_llama-7b.pt"
llama2="../output/final/analysis_meta-llama_Llama-2-7b-hf.pt"

fig=plt.figure(figsize=(5,4))
ax=fig.add_subplot(211)
ax.set_xlabel("params ratio")
ax.set_ylabel("per-type ratio")
# ax.set_title("Llama-7b")

all_block_ratio,all_layer_type_ratio=torch.load(llama2)
target_ratios = list(all_layer_type_ratio.keys())
layer_types = list(all_layer_type_ratio[target_ratios[0]].keys())
for layer_type in layer_types:
    if layer_type=='lm_head':
        continue
    ratios = [np.mean(all_layer_type_ratio[target_ratio][layer_type]) for target_ratio in target_ratios]

    # Line plot
    ax.plot(target_ratios, ratios, marker='o', label=layer_type)

# Adding legend
ax.legend(ncol=2)

ax=fig.add_subplot(212)
ax.set_xlabel("block index")
ax.set_ylabel("per-block ratio")
# ax.set_title("Llama-2-7b")

cnt=0
for target,block_ratio in all_block_ratio.items():
    cnt+=1
    if cnt%2==0:
        continue
    Y=[]
    X=[]
    total_params=0
    for layer_i, ratio_params in block_ratio.items():
        if layer_i=='lm_head':
            continue
        X.append(layer_i)
        total_params=sum([params for ratio,params in ratio_params])
        compressed_params=sum([ratio*params for ratio,params in ratio_params])
        ratio=compressed_params/total_params
        # print(layer_i,ratio)
        Y.append(ratio)
    ax.plot(X,Y,label=f"{target:.2f}")
ax.legend(ncol=2)
ax.ticklabel_format(axis='y',style='plain')
    
# Adding legend
plt.tight_layout()

plt.savefig("../output/final/analysis_meta-llama_Llama-2-7b-hf.pdf",bbox_inches='tight')